#**Segmenting and clustering top venues in some of the neighborhoods of Toronto, Canada**




In [ ]:
# Import essential libraries
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import urlopen
import time
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Obtain the html page

In [ ]:
# Url for obtaining the table consisting of postal codes,
# boroughs and neighbourhoods in Canada
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_page = urlopen(url)

In [ ]:
# BeautifulSoup instance
soup = BeautifulSoup(html_page, 'html.parser')

# Find tables in the html page
tables = soup.find_all('table')

In [ ]:
# The first table in the page corresponds to the
# table consisting of postal codes and neighborhoods
toronto_PC = tables[0]

In [ ]:
# Find the header row of the table
header_row = (toronto_PC.find_all('tr')[0]).find_all('th')

In [ ]:
# Data Rows
data_rows = toronto_PC.find_all('tr')

In [ ]:
# Create a dataframe with columns postal code, borough and neighborhoood
postal_code = []
borough = []
neighborhood = []

for row in data_rows[1:]:
    data_list = [] # data_list is used to store cell value of each row.
    data = row.find_all('td') 
    
    for item in data:
        data_list.append(item.text.strip())
    
    postal_code.append(data_list[0]) # Add postal code to postal_code list
    borough.append(data_list[1]) # Add borough to borough list
    neighborhood.append(data_list[2]) # Add the neighborhood to neighborhood list



In [ ]:
# Create a new dictionary with data collected from the table
postal_codes_dict = {'PostalCode':postal_code , 'Borough' : borough , 'Neighborhood' : neighborhood}

In [ ]:
# Create a dataframe from dictionary
toronto_data = pd.DataFrame(postal_codes_dict)

In [ ]:
# Remove rows having borough as 'Not Assigned'
toronto_data = toronto_data[toronto_data['Borough'] != 'Not assigned']
toronto_data.reset_index(drop=True)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [ ]:
# Shape of resulting dataframe
toronto_data.shape

(103, 3)

#Adding Latitudes and Longitudes for each row

In [ ]:
# Download the csv file of postal codes and respective (latitude,longitude) details.
# Name the file as 'geospatial_data.csv'
!wget -q -O 'geospatial_data.csv' http://cocl.us/Geospatial_data

In [ ]:
# Create a dataframe from csv file downloaded
with open('geospatial_data.csv') as geospatial_json:
    geospatial_data = pd.read_csv(geospatial_json)

In [ ]:
# Show sample of the geospatial dataframe
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
# Make postal code as the index for geospatial dataframe
geospatial_data.index = geospatial_data['Postal Code']

In [ ]:
# Create a new dataframe having postal codes present in 'df' dataframe and
# along with its corresponding latitude and longitude
lat_long_df = geospatial_data.loc[toronto_data['PostalCode'], ['Latitude','Longitude']]
lat_long_df.reset_index(drop = True)
latitude = list(lat_long_df['Latitude'])
longitude = list(lat_long_df['Longitude'])

In [ ]:
# Add the latitude and longitude data to toronto_data df
toronto_data['Latitude'] = latitude
toronto_data['Longitude'] = longitude

In [ ]:
# Checkout the modified dataframe
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#Visulaize the Boroughs/Neighborhoods in map

In [ ]:
# Map instance of Toronto
toronto_map = folium.Map(
    [43.651070,-79.347015],
    zoom_start = 10
)

In [ ]:
# Visualize Map of Toronto
toronto_map

In [ ]:
# Add markers at the boroughs/Neighborhoods
for borough, latitude, longitude in zip(toronto_data['Borough'],toronto_data['Latitude'],toronto_data['Longitude']):
    label = folium.Popup(borough, parse_html = True)
    folium.CircleMarker(
        location=[latitude,longitude],
        radius=10,
        popup=label,
        fill = True
    ).add_to(toronto_map)

In [ ]:
toronto_map

# Segmenting and clustering

In [ ]:
# @hidden_cell
clientId = "EK4UNAQXITYEMJ3UUV3Y5XWMH3H04UQRSCZTSY3AQEPGMXDB"
clientSecret = "CC3YMFZPJJYARNKH1T0G2RGMMQBV1GU3552LFVSZJZHZC3DF"
version = "20180605"

In [ ]:
# Include row in toronto_data where the Borough contins 'Toronto'
toronto_data = toronto_data[toronto_data['Borough'].str.contains('Toronto')]

In [ ]:
# Checkout the new dataframe
toronto_data.reset_index(drop=True, inplace = True)
toronto_data.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [ ]:
# Explore the 5th Neighborhood in the data frame
neighborhood_latitude = toronto_data.loc[4, 'Latitude']
neighborhood_longitude = toronto_data.loc[4, 'Longitude']

neighborhood_name = toronto_data.loc[4, 'Neighborhood']

Find top 100 venues at 'The Beaches' neighborhood within a radius of 1 km

In [ ]:
radius = 1000
limit = 100

foursquare_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(clientId, clientSecret, version,
    neighborhood_latitude, neighborhood_longitude, radius, limit)
foursquare_url

'https://api.foursquare.com/v2/venues/explore?&client_id=EK4UNAQXITYEMJ3UUV3Y5XWMH3H04UQRSCZTSY3AQEPGMXDB&client_secret=CC3YMFZPJJYARNKH1T0G2RGMMQBV1GU3552LFVSZJZHZC3DF&v=20180605&ll=43.67635739999999,-79.2930312&radius=1000&limit=100'

In [ ]:
result_json = requests.get(foursquare_url).json()
result_json

In [ ]:
venues = result_json['response']['groups'][0]['items']

# Venues near the beaches (dataframe)  
nearby_venues = pd.json_normalize(venues)
nearby_venues.head()


In [ ]:
# Method to get venue category
def get_venue_category(dataframe):
    try:
        categories = dataframe['categories']
    except:
        categories = dataframe['venue.categories']
        
    if len(categories) == 0:
        return None
    else:
        return categories[0]['name']

In [ ]:
# List of columns required from dataframe
columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, columns]

# Get category for each venue
nearby_venues['venue.categories'] = nearby_venues.apply(get_venue_category, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

#Explore all neighborhoods of Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        radius = 1000
        limit = 100

        foursquare_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(clientId, clientSecret, version,
                lat, lng, radius, limit)
            
        # Get the json response
        result_json = requests.get(foursquare_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues.append([(
            name, 
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']) for venue in result_json])

    nearby_venues = pd.DataFrame([item for venue_list in venues for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# Nearby venues for each neighborhood
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


In [ ]:
# Group the dataframe by neighborhood
toronto_venues.groupby('Neighborhood').count()

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column 
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 



,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,...,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# toronto_onehot grouped by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,...,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.00,0.000000,0.000000,0.010000,0.0,0.0,0.0,0.0,0.01,0.00,0.000000,0.0,0.00,0.0,0.0,0.010000,0.0,0.030000,0.0,0.000000,0.0,0.0,0.01,0.010000,0.00,0.02,0.0,0.0,0.0,0.010000,0.00,0.0,0.0,0.0,0.020000,0.00000,0.0,0.01,...,0.00,0.0,0.01,0.0,0.0,0.010000,0.0,0.0,0.01,0.010000,0.0,0.0,0.01,0.0,0.00,0.00,0.01,0.0,0.0,0.010000,0.00,0.0,0.00,0.00,0.0,0.000000,0.00,0.01,0.0,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.01,0.000000,0.000000,0.010000,0.0,0.0,0.0,0.0,0.01,0.00,0.020000,0.0,0.01,0.0,0.0,0.000000,0.0,0.040000,0.0,0.050000,0.0,0.0,0.00,0.000000,0.01,0.01,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.010000,0.01000,0.0,0.00,...,0.01,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.02,0.000000,0.0,0.0,0.00,0.0,0.00,0.01,0.01,0.0,0.0,0.000000,0.01,0.0,0.02,0.03,0.0,0.000000,0.01,0.00,0.0,0.0,0.00,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,"Business reply mail Processing Centre, South C...",0.0,0.00,0.000000,0.000000,0.021277,0.0,0.0,0.0,0.0,0.00,0.00,0.021277,0.0,0.00,0.0,0.0,0.021277,0.0,0.042553,0.0,0.021277,0.0,0.0,0.00,0.021277,0.00,0.00,0.0,0.0,0.0,0.021277,0.00,0.0,0.0,0.0,0.021277,0.06383,0.0,0.00,...,0.00,0.0,0.00,0.0,0.0,0.021277,0.0,0.0,0.00,0.042553,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.021277,0.00,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.00,0.066667,0.066667,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.00000,0.0,0.00,...,0.00,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.00,0.0,0.00,0.00,0.0,0.066667,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,Central Bay Street,0.0,0.00,0.000000,0.000000,0.010000,0.0,0.0,0.0,0.0,0.03,0.01,0.020000,0.0,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.00,0.01,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.010000,0.00000,0.0,0.02,...,0.00,0.0,0.00,0.0,0.0,0.010000,0.0,0.0,0.01,0.040000,0.0,0.0,0.00,0.0,0.01,0.00,0.01,0.0,0.0,0.010000,0.02,0.0,0.00,0.00,0.0,0.000000,0.00,0.00,0.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02


In [ ]:
#Print the top 5 venues in each neighborhood
num_top_venues = 5

for neighborhood in toronto_grouped['Neighborhood']:
    print("----" + neighborhood + "----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.12
1                Hotel  0.05
2                 Café  0.05
3  Japanese Restaurant  0.04
4                 Park  0.04


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2   Restaurant  0.06
3          Bar  0.05
4       Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0                  Park  0.09
1           Pizza Place  0.06
2           Coffee Shop  0.06
3               Brewery  0.06
4  Fast Food Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0             Café  0.13
1      Coffee Shop  0.13
2  Harbor / Marina  0.13
3             Park  0.07
4   Scenic Lookout  0.07


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0

In [ ]:
# Method to return the top venues in a neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# Define the max number we need
num_top_venues = 10

# Suffix for 1st,2nd..
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Park,Japanese Restaurant,Restaurant,Bakery,Grocery Store,Gym,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Restaurant,Coffee Shop,Bar,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Sandwich Place,Supermarket
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Brewery,Pizza Place,Sushi Restaurant,Italian Restaurant,Bakery,Fast Food Restaurant,French Restaurant,Flea Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Harbor / Marina,Coffee Shop,Garden,Scenic Lookout,Airport,Airport Lounge,Dog Run,Sculpture Garden,Dance Studio
4,Central Bay Street,Coffee Shop,Sushi Restaurant,Art Gallery,Café,Ramen Restaurant,Park,Japanese Restaurant,Furniture / Home Store,Bubble Tea Shop,Burrito Place


In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 3, 1, 2, 1, 1, 3, 3], dtype=int32)

In [ ]:
# add clustering labels to dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [ ]:
# Merge other details such as latitiude, longitude, borough etc.
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Theater,Diner,Café,Pub,Park,Restaurant,Bakery,Italian Restaurant,Breakfast Spot
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Burger Joint,Café,Ramen Restaurant,Yoga Studio,Men's Store,Bubble Tea Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Gastropub,Japanese Restaurant,Café,Theater,Italian Restaurant,Gay Bar,Diner,Restaurant,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Theater,Italian Restaurant,Gastropub,Hotel,Bakery,Furniture / Home Store,Tea Room
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Pizza Place,Beach,Breakfast Spot,Japanese Restaurant,Park,Caribbean Restaurant,Grocery Store,Bar


In [ ]:
# create map of the clusters
map_clusters = folium.Map(location=[43.651070,-79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters